In [36]:
##  LIBRERIAS  
import os
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from tqdm.notebook import trange, tqdm
import time
import math

import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon

from mpl_toolkits import mplot3d
%matplotlib inline

In [37]:
## LECTURA ... Df Parametros
par_file_name="CLAG1.par"

#print(open(par_file_name, "r").read())
open(par_file_name, "r").close()
l=[re.split("[']", x) for x in open(par_file_name, "r")]
#print(x)
open(par_file_name, "r").close()            #cerrar archivo 
## diccionario de parámetros
param={}
param_l=[]
paran_v=[]

## OJO!! 
# Separador no es el mismo en todos los SO -->  Usar OS

## header:          PARÁMETRO 1
param[1]=l[0][0].strip()
param_l.append(l[0][1].strip().replace(os.sep,""))
## input file:          PARÁMETRO 2
param[2]=l[1][0].strip().rsplit(os.sep, 1)[-1]
param_l.append(l[1][1].strip().replace(os.sep,""))
## columns for data:          PARÁMETRO 3
param[3]=[int(x) for x in l[2][0].strip().split(",")]
param_l.append(l[2][1].strip().replace(os.sep,"")[0:16])
## initial search volume:          PARÁMETRO 4
param[4]=[float(x) for x in l[3][0].strip().split(",")]
param_l.append(l[3][1].strip().replace(os.sep,""))
## Number of Risk Thresholds:          PARÁMETRO 5
param[5]=[int(x) for x in l[4][0].strip().split(",")]
param_l.append(l[4][1].strip().replace(os.sep,"")[0:37])
## Selected risk thresholds:          PARÁMETRO 6
param[6]=[float(x) for x in l[5][0].strip().split(",")]
param_l.append(l[5][1].strip().replace(os.sep,""))
## Number of data in local volume:          PARÁMETRO 7
param[7]=[int(x) for x in l[6][0].strip().split(",")]
param_l.append(l[6][1].strip().replace(os.sep,""))
## Number of units:          PARÁMETRO 8
param[8]=[int(x) for x in l[7][0].strip().split(",")]
param_l.append(l[7][1].strip().replace(os.sep,""))
## Unit names:          PARÁMETRO 9
param[9]=[int(x) for x in l[8][0].strip().split(",")]
param_l.append(l[8][1].strip().replace(os.sep,""))
## not consider values less than:          PARÁMETRO 10
param[10]=[int(x) for x in l[9][0].strip().split(",")]
param_l.append(l[9][1].strip().replace(os.sep,""))
## output files name:          PARÁMETRO 11
param[11]=l[10][0].strip().rsplit(os.sep, 1)[-1]
param_l.append(l[10][1].strip().replace(os.sep,""))
## number of zones:          PARÁMETRO 12
param[12]=[int(x) for x in l[11][0].strip().split(",")]
param_l.append(l[11][1].strip().replace(os.sep,""))
## zone identifiers:          PARÁMETRO 13
param[13]=[int(x) for x in l[12][0].strip().split(",")]
param_l.append(l[12][1].strip().replace(os.sep,""))
## azimuth of search orientation per zone :          PARÁMETRO 14
param[14]=[int(x) for x in l[13][0].strip().split(",")]
param_l.append(l[13][1].strip().replace(os.sep,""))
## Anisotropy ratios:          PARÁMETRO 15
param[15]=[float(x) for x in l[14][0].strip().split(",")]
param_l.append(l[14][1].strip().replace(os.sep,""))
## Exclude data from the same drillhole:          PARÁMETRO 16
param[16]=[int(x) for x in l[15][0].strip().split(",")]
param_l.append(l[15][1].strip().replace(os.sep,""))
## Option to weight data by search expansion:          PARÁMETRO 17
param[17]=[int(x) for x in l[16][0].strip().split(",")]
param_l.append(l[16][1].strip().replace(os.sep,""))

df_parametros = pd.DataFrame(list(zip(list(range(1,18)), param_l, list(param.values()))),
                            columns=['Orden','Descripcion', 'Valor']
                        )


df_parametros






,Orden,Descripcion,Valor
0,1,header,Capping Test Copper Using BHID
1,2,input file,CLAG_MR2022_2.csv
2,3,columns for data,"[1, 2, 3, 4, 6, 5, 7, 8]"
3,4,initial search volume,"[30.0, 30.0, 30.0]"
4,5,Number of Risk Thresholds to consider,[6]
5,6,selected risk thresholds,"[0.1, 0.125, 0.15, 0.175, 0.2, 0.225]"
6,7,number of data in local volume,[21]
7,8,number of units,[2]
8,9,unit names,"[31, 32]"
9,10,do not consider values less than,[0]


In [38]:
##   Lectura CSV compositos .. validar si hay nulos o no

df_CLAG_MR_2022 = pd.read_csv(param[2])
df_CLAG_MR_2022.head(2)
## Requerimiento: No NaN ni missing values
'''
Algoritmo localcapping no admite valores nulos ni missings
función missing_val(df) identifica casos missing y nulos
'''

def missing_val(df):
    cero_val = (df == 0.00).astype(int).sum(axis=0)
    mis_val = df.isnull().sum()
    mz_table = pd.concat([cero_val, mis_val], axis=1)
    mz_table = mz_table.rename(
               columns = {0 : 'Valores  Cero', 1 : 'Valores Missing'})
    mz_table['Total Cero + Missing'] = mz_table['Valores  Cero'] + mz_table['Valores Missing']
    mz_table = mz_table[mz_table.iloc[:,1] != 0].round(1)
    print ("Df Columnas: " + str(df.shape[1]) + " Df Filas: " + str(df.shape[0]) + " \n"      
            "Columnas con Ceros / Missing: " + str(mz_table.shape[0]))
    return mz_table

## 
missing_val(df_CLAG_MR_2022)  




Df Columnas: 8 Df Filas: 4255 
Columnas con Ceros / Missing: 0


,Valores Cero,Valores Missing,Total Cero + Missing


In [39]:
df_CLAG_MR_2022.head(2)

,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID
0,2336,4072.2950,4850.2450,4199.8669,25.146513,32,42,A019
1,2337,4073.5668,4848.7522,4196.7885,40.000000,31,42,A019


In [40]:
# ACONDICIONAR DATAFRAME


'''
Este ordenamiento no limita la seleccion decolumnas por nombre
Hace seleccion en funcion del orden inicial de columnas
'''
df_CLAG_MR_2022_R=df_CLAG_MR_2022[[df_CLAG_MR_2022.iloc[:,0].name,
                                   df_CLAG_MR_2022.iloc[:,1].name,
                                   df_CLAG_MR_2022.iloc[:,2].name,
                                   df_CLAG_MR_2022.iloc[:,3].name,
                                   df_CLAG_MR_2022.iloc[:,4].name,
                                   df_CLAG_MR_2022.iloc[:,5].name,
                                   df_CLAG_MR_2022.iloc[:,6].name,
                                   df_CLAG_MR_2022.iloc[:,7].name]]

df_CLAG_MR_2022=df_CLAG_MR_2022_R

df_CLAG_MR_2022.head(2)


##  $$$  INICILIZAR VALORES PARA 6 UMBRALES
##  y    #ID,BHID,X,Y,Z,CODE,ZONE,VALUE,CAPVAL1,CAPPED1,CAPVAL2,CAPPED2,CAPVAL3,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6
#         ,AvgSin,PROP_12,Num_Data

df_CLAG_MR_2022["CAPVAL1"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED1"]=0
df_CLAG_MR_2022["CAPVAL2"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED2"]=0
df_CLAG_MR_2022["CAPVAL3"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED3"]=0
df_CLAG_MR_2022["CAPVAL4"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED4"]=0
df_CLAG_MR_2022["CAPVAL5"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED5"]=0
df_CLAG_MR_2022["CAPVAL6"]=df_CLAG_MR_2022.iloc[:,4]
df_CLAG_MR_2022["CAPPED6"]=0

df_CLAG_MR_2022["AvgSin"]=0
df_CLAG_MR_2022["PROP_12"]=0
df_CLAG_MR_2022["Num_Data"]=0

df_CLAG_MR_2022_ORIG=df_CLAG_MR_2022

df_CLAG_MR_2022.head(2)






,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,CAPVAL1,CAPPED1,...,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
0,2336,4072.2950,4850.2450,4199.8669,25.146513,32,42,A019,25.146513,0,...,0,25.146513,0,25.146513,0,25.146513,0,0,0,0
1,2337,4073.5668,4848.7522,4196.7885,40.000000,31,42,A019,40.000000,0,...,0,40.000000,0,40.000000,0,40.000000,0,0,0,0


In [41]:
df_parametros.Valor[3][0]

30.0

In [42]:
df_parametros.Valor[14]

[2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5]

In [43]:


parametro_expansion=0.9
expansiones_max=12
expansiones_realizadas=0
round_hemi_accuracy=2

max_compositos_interiores=19  # ( restar 1 al numero objetivo)

freno_1_composito_x_roca=0  ###  SOLO LEER UN SOLO COMPOSITO POR COMBINACION ROCA/ZONA
freno_2_roca_x_zona=0  ###  SOLO LEER UNA SOLA ROCA POR ZONA
freno_3_zona=0    ### SOLO LEER UNICA ZONA

iterador=0      ##  PARA SIGNAR RATIO x ZONA

flag_holeid=df_parametros.Valor[15][0]

#for zone in df_parametros.Valor[12][:1]:     ## borrar   [:2]
zonas = df_parametros.Valor[12]
for zone in zonas:     #################################  FIJO PARA PRUEBA  BORRAR
    
    print("\n\n")
    print("zone:",zone)
    print(" ") 
    #  df_parametros.Valor[13]         ## LISTA DE AZIMUTH
    print("azimuth:",df_parametros.Valor[13][iterador])
    azimuth=df_parametros.Valor[13][iterador]
    cartesiana=0          ## <------ 

    if azimuth < 90:
        cartesiana=90-azimuth 
    elif azimuth < 180:
        cartesiana=360-(azimuth-90) 
    elif azimuth < 270:
        cartesiana=270-(azimuth-180) 
    elif azimuth < 360:
        cartesiana=180-(azimuth-270) 
        
    print("cartesiana:",cartesiana)
    rad=math.radians(cartesiana)    ## <------  APLICAR ACA SIN COS etc etc    
    print("radian:",rad)
    #df_parametros.Valor[14]                            ## LISTA DE ANISOTROPIA 
    print("ratio anisotropia:",df_parametros.Valor[14][iterador])

    ratio=df_parametros.Valor[14][iterador]
    hemi_axis_x, hemi_axis_y, hemi_axis_z = [], [], []
    for i in range(expansiones_max):       #  30*30*30       30*75*30
        # print((df_parametros.Valor[3][0] / 2)*((1+parametro_expansion)**i))
        # print(((df_parametros.Valor[3][1]*ratio) / 2)*((1+parametro_expansion)**i))
        # print((df_parametros.Valor[3][2] / 2)*((1+parametro_expansion)**i))
        # print((df_parametros.Valor[3][0] / 2)*((1+parametro_expansion)**i)*2)
        # print(((df_parametros.Valor[3][1]*ratio) / 2)*((1+parametro_expansion)**i)*2)
        # print((df_parametros.Valor[3][2] / 2)*((1+parametro_expansion)**i)*2)
        semi_x=(df_parametros.Valor[3][0] / 2)*((1+parametro_expansion)**i)
        semi_y=((df_parametros.Valor[3][1]*ratio) / 2)*((1+parametro_expansion)**i)
        semi_z=(df_parametros.Valor[3][2] / 2)*((1+parametro_expansion)**i)
        hemi_axis_x.append(round(semi_x,round_hemi_accuracy))
        hemi_axis_y.append(round(semi_y,round_hemi_accuracy))
        hemi_axis_z.append(round(semi_z,round_hemi_accuracy))

    ###for rock in df_parametros.Valor[8][:1]:## borrar   [3]
    rocks=df_parametros.Valor[8]
    for rock in rocks:

        freno_1_composito_x_roca=0

        print("\n\n\n")
        print("    rock:",rock)
        print("    zone:",zone)
        print(" ")
        print("    parametros de la Roca ")
        print("    ratio anisotropia:",ratio)
        print("    azimuth: ",azimuth)
        print("    cartesiana: ",cartesiana)
        print("    radian: ",rad)

        for obs in df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T:  ## ITERAR INDICES
            print("\n")
            print("        index composito :",obs)
            print(" ")
            x=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,1][obs]
            y=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,2][obs]
            z=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,3][obs]
             
            print("        x: ",x)
            print("        y: ",y)
            print("        z: ",z)
            obs_holeid=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[obs][7]
            print(" ")
            print("        holeid: ",obs_holeid)
            # x_SI,y_SI= x - semi_x,y + semi_y        
            # x_II,y_II= x - semi_x,y - semi_y
            # x_SD,y_SD= x + semi_x,y + semi_y
            # x_ID,y_ID= x + semi_x,y - semi_y      
            # print("        SI:",x_SI,y_SI)
            # print("        II:",x_II,y_II)  
            # print("        SD:",x_SD,y_SD) 
            # print("        ID:",x_ID,y_ID)
            # s = gpd.GeoSeries([Polygon([(x_II,y_II), (x_SI, y_SI), (x_SD, y_SD), (x_ID, y_ID)])])      
            ## RESPETAR ORDEN II SI SD ID
            #s[0]
            #s.rotate(cartesiana, origin=(x,y))[0]
            # rx,ry = s.rotate(  cartesiana, origin=(x,y))[0].exterior.coords.xy
            # coords = np.dstack((rx,ry)).tolist()
            #print(coords)           
            # print("        Rotated coords: ")
            # print("        ",coords[0][0][0],coords[0][0][1])
            # print("        ",coords[0][1][0],coords[0][1][1])
            # print("        ",coords[0][2][0],coords[0][2][1])
            # print("        ",coords[0][3][0],coords[0][3][1])
            # print("        ",coords[0][4][0],coords[0][4][1])
            # rotado2d=s.rotate(cartesiana, origin=(x,y))[0]
            # index_inner_3d=[]## INDICES EN POLIGONO ROTADO 3D  
            index_inner_3d=[]## INDICES EN POLIGONO ROTADO 3D
            ley=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,4][obs]
            print("        ley composito : ",ley)

            for i in range(expansiones_max):
                print(" ")
                print(" ")
                print("            expansion: ",i)
                print(" ")
                print("            semi eje x:",hemi_axis_x[i])
                print("            semi eje y:",hemi_axis_y[i])
                print("            semi eje z:",hemi_axis_z[i])

                x_SI,y_SI= x - hemi_axis_x[i] , y + hemi_axis_y[i]       
                x_II,y_II= x - hemi_axis_x[i] , y - hemi_axis_y[i]
                x_SD,y_SD= x + hemi_axis_x[i] , y + hemi_axis_y[i]
                x_ID,y_ID= x + hemi_axis_x[i] , y - hemi_axis_y[i]  

                print("            SI:  x: ",x_SI,"  y:",y_SI)
                print("            II:  x: ",x_II,"  y:",y_II)  
                print("            SD:  x: ",x_SD,"  y:",y_SD) 
                print("            ID:  x: ",x_ID,"  y:",y_ID)

                s = gpd.GeoSeries([Polygon([(x_II,y_II), (x_SI, y_SI), (x_SD, y_SD), (x_ID, y_ID)])])      
                # RESPETAR ORDEN II SI SD ID
                #s[0]
                #s.rotate(cartesiana, origin=(x,y))[0]
                #rx,ry = s.rotate(  cartesiana, origin=(x,y))[0].exterior.coords.xy
                rotado2d=s.rotate(cartesiana, origin=(x,y))[0]
                rx,ry = rotado2d.exterior.coords.xy
                coords = np.dstack((rx,ry)).tolist()
                print("            Rotated coords: ")
                print("            x: ",coords[0][0][0]," y: ",coords[0][0][1])
                print("            x: ",coords[0][1][0]," y: ",coords[0][1][1])
                print("            x: ",coords[0][2][0]," y: ",coords[0][2][1])
                print("            x: ",coords[0][3][0]," y: ",coords[0][3][1])
                print("            x: ",coords[0][4][0]," y: ",coords[0][4][1])


                for obs_sub in df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T:
                    print(" ")
                    print("                obs    : ",obs)
                    print("                obs_sub: ",obs_sub)
                    sub_ley=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].iloc[:,4][obs_sub]
                    print("                ley_sub: ",sub_ley)    

                    x_int=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[obs_sub][1]
                    y_int=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[obs_sub][2]
                    print("                x_sub: ",x_int)
                    print("                y_sub: ",y_int)

                    composito_inter= gpd.GeoSeries([Point(x_int,y_int)])  ##  COMPONER coords del subcomposito

                    if rotado2d.intersects(composito_inter[0]) == True:
                        print("                    SI 2D")

                        z_int=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[obs_sub][3]
                        
                        print("                    z_sub: ",z_int)
                        print("                    z:     ",z)
                        z_inner_diff=abs(z-z_int)
                        print("                    dist z:",z_inner_diff)
                        print("                    dist max z:",hemi_axis_z[i])

                        if z_inner_diff <= hemi_axis_z[i]:
                            print("                        SI 3D")
                            #index_inner_3d.append(obs_sub)
                            #print("                        index_inner_3d: ",index_inner_3d)
                            print("                        drillhole obs    : ",obs_holeid)
                            subobs_holeid=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[obs_sub][7]
                            print("                        drillhole obs sub: ",subobs_holeid)

                            if obs_sub not in index_inner_3d:
                                if flag_holeid==1:
                                    print("                            SI FLAG EXCLUIR SAME ID")
                                    if obs_holeid!=subobs_holeid:
                                        print("                            DISTINTO ID  -->  APPEND") 
                                        index_inner_3d.append(obs_sub)
                                else:
                                    index_inner_3d.append(obs_sub)

                    else:
                        print("                    NO 2D") 


                    if len(index_inner_3d) > max_compositos_interiores:       ####  ROMPER APENAS ENCUENTRE 20
                        break

                print("\n\n")
                print(index_inner_3d)
                print("longitud index_inner:   ",len(index_inner_3d))
                print("# expansion sub compositos:   ",i)
                ## NECESARIO ROMPER A ESTE NIVEL 
                if len(index_inner_3d) > max_compositos_interiores:   ###  ROMPER EXPANSION
                    print("# expansion break bucles expansivos:   ",i)
                    break   ###  ROMPER EXPANSION

                # if i > 8:   ###  ROMPER EXPANSION por numero de expansiones
                #     break   ###  ROMPER EXPANSION
            print(" ")
            print("verificar lista de busqueda...")
            print("longitud inder_inner fuera del bucle    ",len(index_inner_3d))
 
            print("indice composito en evaluacion -- NO SUB--   ",obs)
            index_inner_3d.append(obs)
            print("lista final de compositos SUB asociados a composito en evaluacion + composito en evaluacion")
            print(index_inner_3d)
            print("longitud final index  ",len(index_inner_3d))

            df_CLAG_MR_2022.T[obs][22]==len(index_inner_3d)
            df_CLAG_MR_2022.T[obs][20]==sub_df.iloc[:,0].mean()

            print(" ")
            print(" ")
            print("Dataframe solo inner index")
            print(df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[index_inner_3d])
            print("solo leyes")
            print(df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[index_inner_3d].iloc[[4]])
            print("\n")

            sub_df=df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[index_inner_3d].iloc[[4]].T
            print("\n")        
            print(sub_df)
            print("\n")

            sub_df["percent_ley"] = (sub_df.iloc[:,0] / sub_df.iloc[:,0].sum()) 
            print("\n")
            print(sub_df)
            print("\n")
            print("  Umbrales",df_parametros.Valor[5])
            print("\n")
            print("ley promedio")
            print(sub_df.iloc[:,0].mean())
            print("\n")
            print("relativo aportado por el composito ",sub_df.iloc[-1,1]*100,"%")  ## procentaje del composito
            print(" ")
            print(" ")      

            

            print("pre_bucle")

            ley_sin=sub_df.iloc[:-1,1].mean()      ### promedio SIN composito central
            ley_con=sub_df.iloc[:,1].mean()        ### promedio CON composito central
            variax=abs((ley_con/ley_sin)-1)

            if variax > df_parametros.Valor[5][0]:
                print("1")
                print(df_parametros.Valor[5][0])
                df_CLAG_MR_2022.iloc[obs,8]=sub_df.iloc[:,0].mean()
                df_CLAG_MR_2022.iloc[obs,9]=1
                
                if variax > df_parametros.Valor[5][1]:
                    print("2")
                    print(df_parametros.Valor[5][1])
                    df_CLAG_MR_2022.iloc[obs,10]=sub_df.iloc[:,0].mean()
                    df_CLAG_MR_2022.iloc[obs,11]=1         
                    if variax> df_parametros.Valor[5][2]:
                        print("3")
                        print(df_parametros.Valor[5][2])
                        df_CLAG_MR_2022.iloc[obs,12]=sub_df.iloc[:,0].mean()
                        df_CLAG_MR_2022.iloc[obs,13]=1
                        if variax > df_parametros.Valor[5][3]:
                            print("4")
                            print(df_parametros.Valor[5][3])
                            df_CLAG_MR_2022.iloc[obs,14]=sub_df.iloc[:,0].mean()
                            df_CLAG_MR_2022.iloc[obs,15]=1
                            if variax > df_parametros.Valor[5][4]:
                                print("5")
                                print(df_parametros.Valor[5][4])
                                df_CLAG_MR_2022.iloc[obs,16]=sub_df.iloc[:,0].mean()
                                df_CLAG_MR_2022.iloc[obs,17]=1
                                if variax > df_parametros.Valor[5][5]:
                                    print("6")
                                    print(df_parametros.Valor[5][5])
                                    df_CLAG_MR_2022.iloc[obs,18]=sub_df.iloc[:,0].mean()
                                    df_CLAG_MR_2022.iloc[obs,19]=1               
            else:
                print("NO EN UMBRAL MINIMO")

            
            freno_1_composito_x_roca=freno_1_composito_x_roca+1
            if freno_1_composito_x_roca>10:
                break
        freno_2_roca_x_zona=freno_2_roca_x_zona+1
        if freno_2_roca_x_zona>1:
            break
    freno_3_zona=freno_3_zona+1
    if freno_3_zona>0:
        break
    iterador=iterador+1


    print(" ")
    print(" ")







zone: 41
 
azimuth: 140
cartesiana: 310
radian: 5.410520681182422
ratio anisotropia: 2.5




    rock: 31
    zone: 41
 
    parametros de la Roca 
    ratio anisotropia: 2.5
    azimuth:  140
    cartesiana:  310
    radian:  5.410520681182422


        index composito : 73
 
        x:  3185.3567
        y:  4329.8712
        z:  4116.0064
 
        holeid:  A055
        ley composito :  0.828571
 
 
            expansion:  0
 
            semi eje x: 15.0
            semi eje y: 37.5
            semi eje z: 15.0
            SI:  x:  3170.3567   y: 4367.3712
            II:  x:  3170.3567   y: 4292.3712
            SD:  x:  3200.3567   y: 4367.3712
            ID:  x:  3200.3567   y: 4292.3712
            Rotated coords: 
            x:  3146.98821923774  y:  4317.257331283538
            x:  3204.4415524716633  y:  4365.466402010029
            x:  3223.7251807622597  y:  4342.48506871646
            x:  3166.2718475283364  y:  4294.275997989969
            x:  3146.98821923774  

In [34]:
ag_df=pd.read_csv("C:\FORTRAN\clsilver_mr20221.out",sep=",")
ag_df[ag_df.ID==2336]

,ID,BHID,X,Y,Z,CODE,ZONE,VALUE,CAPVAL1,CAPPED1,...,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
0,2336,A019,4072.29,4850.25,4199.87,32,42.0,25.14651,25.14651,0,...,0,25.14651,0,25.14651,0,25.14651,0,30.6503,0.231,21


In [9]:
df_CLAG_MR_2022

,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,CAPVAL1,CAPPED1,...,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
0,2336,4072.2950,4850.2450,4199.8669,25.146513,32,42,A019,25.146513,0,...,0,25.146513,0,25.146513,0,25.146513,0,0,0,0
1,2337,4073.5668,4848.7522,4196.7885,40.000000,31,42,A019,40.000000,0,...,0,40.000000,0,40.000000,0,40.000000,0,0,0,0
2,2338,4074.6121,4847.5252,4194.2584,35.799999,31,42,A019,35.799999,0,...,0,35.799999,0,35.799999,0,35.799999,0,0,0,0
3,2339,4075.6575,4846.2982,4191.7282,44.693333,31,42,A019,44.693333,0,...,0,44.693333,0,44.693333,0,44.693333,0,0,0,0
4,2340,4076.7028,4845.0713,4189.1980,45.193333,31,42,A019,45.193333,0,...,0,45.193333,0,45.193333,0,45.193333,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,390969,4234.5329,5190.9378,4185.3258,7.000000,31,42,A913,7.000000,0,...,0,7.000000,0,7.000000,0,7.000000,0,0,0,0
4251,390970,4234.9222,5190.5647,4181.7917,7.457831,31,42,A913,7.457831,0,...,0,7.457831,0,7.457831,0,7.457831,0,0,0,0
4252,390971,4235.3126,5190.1954,4178.2573,8.333333,31,42,A913,8.333333,0,...,0,8.333333,0,8.333333,0,8.333333,0,0,0,0
4253,394849,3690.3063,4739.9698,4152.7452,122.400002,32,41,A951,122.400002,0,...,0,122.400002,0,122.400002,0,122.400002,0,0,0,0


In [85]:
sub_df.iloc[-1,1]

0.003079128756818603

In [84]:
df_parametros.Valor[5][0]

0.1

In [27]:
#########################################
do irt=1,numrisk
    icap(irt)=0
    valcap(irt)=grade(id)

            avgwith=sumwith/(sumwt+1.)
            avgwo=sumwo/real(sumwt)
            varwo=sumsqwo/real(sumwt)-avgwo*avgwo
            ratio=(avgwith-avgwo)/avgwo

            if(ratio.gt.RiskT(irt)) then
    valcap(irt)=sumwo*((real(sumwt+1)/real(sumwt))*(1+RiskT(irt))-1)
icap(irt)=1
            endif
if(idbug.eq.1.and.irecnum(id).ge.idbs.and.irecnum(id).le.idbe)
1  then
write(8,*)
write(8,*) "Capping Results, Risk Level ",RiskT(irt)
write(8,*) "ID,Avg With, Avg Without, Ratio Grade, Grade Cap"
write(8,1243)
1   irecnum(id),avgwith,avgwo,ratio,grade(id),valcap(irt)		

###########





,Ag_ppm_BESTEL,percent_ley
377,9.0,0.033446
378,4.6,0.017094
369,18.333334,0.06813
370,11.266666,0.041869
371,7.0,0.026013
372,29.75,0.110557
373,17.533333,0.065157
374,28.416666,0.105602
375,22.666666,0.084234
376,43.129032,0.160276


In [22]:
sub_df.iloc[-1,1]

0.003079128756818603

In [13]:
for obs in df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T:  ## ITERAR INDICES
    print("        index composito :",obs)

        index composito : 70
        index composito : 71
        index composito : 72
        index composito : 78
        index composito : 79
        index composito : 80
        index composito : 81
        index composito : 82
        index composito : 83
        index composito : 84
        index composito : 85
        index composito : 86
        index composito : 87
        index composito : 88
        index composito : 89
        index composito : 90
        index composito : 91
        index composito : 92
        index composito : 93
        index composito : 94
        index composito : 95
        index composito : 96
        index composito : 97
        index composito : 98
        index composito : 99
        index composito : 100
        index composito : 101
        index composito : 102
        index composito : 103
        index composito : 104
        index composito : 105
        index composito : 106
        index composito : 107
        index composito : 108
     

In [ ]:
for obs in df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T:  ## ITERAR INDICES
    print("        index composito :",obs)

In [11]:
df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T

,70,71,72,78,79,80,81,82,83,84,...,4237,4238,4239,4240,4241,4242,4243,4244,4253,4254
COUNT,6583,6584,6585,6591,6592,6593,6594,6595,6596,6597,...,386876,386877,386882,386883,386884,386885,386886,386887,394849,394850
mid_x,3180.6759,3182.2365,3183.6408,3193.1512,3194.7091,3196.2666,3197.8238,3199.3806,3200.9371,3202.4932,...,3630.0933,3631.2403,3637.5617,3638.8036,3640.0458,3641.2863,3642.5218,3643.6524,3690.3063,3690.807
mid_y,4334.9154,4333.2336,4331.7203,4321.4715,4319.7927,4318.1143,4316.4362,4314.7585,4313.0812,4311.4043,...,4810.1368,4809.076,4803.2004,4802.0486,4800.899,4799.7462,4798.5894,4797.5269,4739.9698,4739.4052
mid_z,4121.8069,4119.874,4118.1339,4106.3269,4104.3892,4102.4509,4100.512,4098.5726,4096.6325,4094.6918,...,4030.9771,4028.7139,4016.141,4013.6649,4011.1879,4008.7116,4006.2347,4003.9641,4152.7452,4149.3531
Ag_ppm_BESTEL,0.348333,0.15,0.74375,11.04,28.060001,22.67,7.3,19.933332,12.213333,1.823333,...,28.0,17.0,109.0,76.0,19.0,24.0,18.0,14.0,122.400002,47.506329
NBKRT,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
AG_SUBZONAS,41,41,41,41,41,41,41,41,41,41,...,41,41,41,41,41,41,41,41,41,41
HOLEID,A055,A055,A055,A055,A055,A055,A055,A055,A055,A055,...,A858,A858,A858,A858,A858,A858,A858,A858,A951,A951
CAPVAL1,0.348333,0.15,0.74375,11.04,28.060001,22.67,7.3,19.933332,12.213333,1.823333,...,28.0,17.0,109.0,76.0,19.0,24.0,18.0,14.0,122.400002,47.506329
CAPPED1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_CLAG_MR_2022[(df_CLAG_MR_2022.iloc[:, 5] == rock) & (df_CLAG_MR_2022.iloc[:, 6] == zone)].T[70]

COUNT                 6583
mid_x            3180.6759
mid_y            4334.9154
mid_z            4121.8069
Ag_ppm_BESTEL     0.348333
NBKRT                   32
AG_SUBZONAS             41
HOLEID                A055
CAPVAL1           0.348333
CAPPED1                  0
CAPVAL2           0.348333
CAPPED2                  0
CAPVAL3           0.348333
CAPPED3                  0
CAPVAL4           0.348333
CAPPED4                  0
CAPVAL5           0.348333
CAPPED5                  0
CAPVAL6           0.348333
CAPPED6                  0
AvgSin                   0
PROP_12                  0
Num_Data                 0
Name: 70, dtype: object

In [ ]:
df.set_index('month')

In [62]:
sub_df.iloc[:,0]

377           9.0
378           4.6
369     18.333334
370     11.266666
371           7.0
372         29.75
373     17.533333
374     28.416666
375     22.666666
376     43.129032
1626        13.95
1627         13.0
1628    18.985075
1629         12.6
1632          8.0
1633          1.2
1634          3.0
1644     2.166667
1645     2.666667
1646          1.0
73       0.828571
Name: Ag_ppm_BESTEL, dtype: object

In [61]:
sub_df.iloc[:,0].mean()

12.813937000000001

In [20]:
df_test

,COUNT,mid_x,mid_y,mid_z,Ag_ppm_BESTEL,NBKRT,AG_SUBZONAS,HOLEID,CAPVAL1,CAPPED1,...,CAPPED3,CAPVAL4,CAPPED4,CAPVAL5,CAPPED5,CAPVAL6,CAPPED6,AvgSin,PROP_12,Num_Data
0,2336,4072.2950,4850.2450,4199.8669,25.146513,32,42,A019,25.146513,0,...,0,25.146513,0,25.146513,0,25.146513,0,0,0,0
1,2337,4073.5668,4848.7522,4196.7885,40.000000,31,42,A019,40.000000,0,...,0,40.000000,0,40.000000,0,40.000000,0,0,0,0
2,2338,4074.6121,4847.5252,4194.2584,35.799999,31,42,A019,35.799999,0,...,0,35.799999,0,35.799999,0,35.799999,0,0,0,0
3,2339,4075.6575,4846.2982,4191.7282,44.693333,31,42,A019,44.693333,0,...,0,44.693333,0,44.693333,0,44.693333,0,0,0,0
4,2340,4076.7028,4845.0713,4189.1980,45.193333,31,42,A019,45.193333,0,...,0,45.193333,0,45.193333,0,45.193333,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,390969,4234.5329,5190.9378,4185.3258,7.000000,31,42,A913,7.000000,0,...,0,7.000000,0,7.000000,0,7.000000,0,0,0,0
4251,390970,4234.9222,5190.5647,4181.7917,7.457831,31,42,A913,7.457831,0,...,0,7.457831,0,7.457831,0,7.457831,0,0,0,0
4252,390971,4235.3126,5190.1954,4178.2573,8.333333,31,42,A913,8.333333,0,...,0,8.333333,0,8.333333,0,8.333333,0,0,0,0
4253,394849,3690.3063,4739.9698,4152.7452,122.400002,32,41,A951,122.400002,0,...,0,122.400002,0,122.400002,0,122.400002,0,0,0,0


In [27]:
df_test=df_test.set_index("COUNT")

KeyError: "None of ['COUNT'] are in the columns"

In [68]:
sub_df.iloc[:,1]
sub_df.iloc[:,1].mean()

0.04761904761904761

In [69]:
sub_df.iloc[:,1]

377     0.033446
378     0.017094
369      0.06813
370     0.041869
371     0.026013
372     0.110557
373     0.065157
374     0.105602
375     0.084234
376     0.160276
1626    0.051841
1627     0.04831
1628    0.070552
1629    0.046824
1632     0.02973
1633    0.004459
1634    0.011149
1644    0.008052
1645     0.00991
1646    0.003716
73      0.003079
Name: percent_ley, dtype: object

In [83]:
ley_sin=sub_df.iloc[:-1,1].mean()      ### promedio SIN composito central
ley_con=sub_df.iloc[:,1].mean()        ### promedio CON composito central
variax=abs((ley_con/ley_sin)-1)




0.044677486595989135

In [56]:
df_CLAG_MR_2022.T[73][22] 

0

In [57]:
df_CLAG_MR_2022.iloc[obs,8]

0.828571

In [60]:
df_CLAG_MR_2022.T[obs][8] 

0.828571

In [86]:
df_CLAG_MR_2022.iloc[obs,8]

0.828571

In [88]:
df_CLAG_MR_2022.iloc[obs,9]

0